In [3]:
%load_ext autoreload

In [ ]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

import loadBar
from csv_parser import CSVParser
from globals import RESOURCE_FOLDER
from markovSquares import MarkovSquares
from data_handler import LocalToLargeDataLoader
from searoutePointFinder import fill_with_proximity


In [ ]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../resources")


In [ ]:
index_data = parsed_data.copy()
index_data.set_index("time", inplace=True)

In [ ]:
def resampler(df, sorting_column, freq):
    unique_ids = df[sorting_column].unique()
    final_df = pd.DataFrame()
    partial_list = []

    for i in range(len(unique_ids)):
        loadBar.load_bar(len(unique_ids),i+1)
        resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()

        resample_partial = fill_with_proximity(resample_partial)
        partial_list.append(resample_partial)

    for chunk in partial_list:
        final_df = pd.concat([final_df,chunk])
    
    return final_df

print(index_data)
resampled_data_20min = resampler(index_data, "vesselId", "20min")

resampled_data_20min.to_csv("../../resources/data_resampled_20min.csv")

In [ ]:
print(resampled_data_20min)

In [ ]:
class MarkovSquares():

    def __init__(self, square_size):
        self.square_size = square_size
        self.normalized_markov_matrix = None
        self.direction_columns = ["SW", "S", "SE", "W", "C", "E", "NW", "N", "NE"]

    def markovSquares(self, df, sorting_column):
        unique_sorts = df[sorting_column].unique()

        markov_matrix = np.zeros((180//self.square_size,360//self.square_size,9))
        print("Generating Markov Matrix")
        for j, sorts in enumerate(unique_sorts):
            
            loadBar.load_bar(len(unique_sorts),j)
            sort_df = df[df[sorting_column] == sorts]

            #Iterate through all columns for input features
            for i in range(len(sort_df)-1):
                entry1 = sort_df.iloc[i]
                entry2 = sort_df.iloc[i+1]

                lat1 = entry1["latitude"]
                lon1 = entry1["longitude"]

                lat2 = entry2["latitude"]
                lon2 = entry2["longitude"]

                lat_diff = (lat1-lat2)//self.square_size
                lon_diff = (lon1-lon2)//self.square_size

                inner_idx=self.markov_index_inner(lat_diff, lon_diff)

                lat_idx=int(lat1//self.square_size)
                lon_idx=int(lon1//self.square_size)
            
                markov_matrix[lat_idx][lon_idx][inner_idx]+=1
            
        self.normalized_markov_matrix  = markov_matrix / np.sum(markov_matrix, axis=-1, keepdims=True)

    def add_as_columns(self, df):
        # Define column names for the new data
        column_names = self.direction_columns
        
        # Initialize a DataFrame to hold the results
        results_df = pd.DataFrame(columns=column_names, index=df.index)
        
        print("Adding Markov Squares as columns")
        i=0
        for idx, row in df.iterrows():
            i+=1
            loadBar.load_bar(len(df),i)
            # Extract latitude and longitude values
            latitude = row["latitude"]
            longitude = row["longitude"]
            
            # Check for NaN values in latitude and longitude
            if pd.notna(latitude) and pd.notna(longitude):
                # Get the processed values (assumes `get_markov_square` returns a list of 9 values)
                processed_values = self.get_markov_square(latitude, longitude)
                # Set the result in the corresponding row of results_df
                results_df.loc[idx] = processed_values
            else:
                # Set NaN for each new column if latitude or longitude is NaN
                results_df.loc[idx] = [np.nan] * 9

        # Concatenate results_df with the original df along the columns axis
        df = pd.concat([df, results_df], axis=1)
        
        return df


    def get_markov_square(self, lat, lon):
        lat_idx = int(lat//self.square_size)
        lon_idx = int(lon//self.square_size)
        return self.normalized_markov_matrix[lat_idx][lon_idx]

    def markov_index_inner(self, lat_diff, lon_diff):
        lat_index = 0 if lat_diff <= -1 else (1 if lat_diff == 0 else 2)
        lon_index = 0 if lon_diff <= -1 else (1 if lon_diff == 0 else 2)
        index = lat_index * 3 + lon_index
        return index

In [ ]:
total_df = pd.read_csv(RESOURCE_FOLDER+"/data_resampled_20min.csv")


# total_df = pd.read_csv(RESOURCE_FOLDER+"/resampled_data_h.csv")

total_df['etaParsed'] = pd.to_datetime(total_df['etaParsed'])
total_df["time"] = pd.to_datetime(total_df['time'])

start_date = pd.to_datetime('2024-01-01')

total_df["etaParsed"] = (total_df['etaParsed'] - start_date).dt.days

In [ ]:
markov = MarkovSquares(2)
markov.markovSquares(total_df, "vesselId")
np.save(markov.normalized_markov_matrix, RESOURCE_FOLDER+"/markov_matrix.npy")

In [ ]:
total_df = markov.add_as_columns(total_df)

In [ ]:

time_diffs = total_df["time"].diff()
time_interval = time_diffs.dropna().iloc[0]
time_interval = int(time_interval.total_seconds()/(60*20))



total_df.set_index("time", inplace=True)

In [ ]:
total_df.to_csv(RESOURCE_FOLDER+"/data_resampled_20min_markov.csv")

In [ ]:
print(len(total_df))

In [ ]:
total_df = pd.read_csv(RESOURCE_FOLDER+"/data_resampled_20min_markov.csv")

In [5]:
print(total_df.head())

                  time    cog  sog  rot  heading  navstat   latitude  \
0  2024-01-01 00:00:00  284.0  0.7  0.0     88.0      0.0 -34.743700   
1  2024-01-01 00:20:00  284.0  0.7  0.0     88.0      0.0 -34.627229   
2  2024-01-01 00:40:00  284.0  0.7  0.0     88.0      0.0 -34.510757   
3  2024-01-01 01:00:00  284.0  0.7  0.0     88.0      0.0 -34.394286   
4  2024-01-01 01:20:00  284.0  0.7  0.0     88.0      0.0 -34.277814   

   longitude                  vesselId                    portId  ...  \
0 -57.851300  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  ...   
1 -57.966135  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  ...   
2 -58.080969  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  ...   
3 -58.195804  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  ...   
4 -58.310639  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  ...   

   portLatitude        SW         S   SE         W         C    E   NW    N  \
0      -33.5875  0.078325  0.485714  0.0  0.34803

In [6]:
STEPSIZES = [1] #3, 6, 18, 36, 72, 144, 216, 288, 360]
OUTPUT_WINDOW = 1
INPUT_WINDOW = 4
OUTPUT_FORECAST = ["latitude", "longitude", "cog", "sog", "rot", "heading"]

In [7]:
#Make time series into supervised problem

# 1 = 20 minutes
# 3 = 1 hour
# 18 = 6 hours
# 72 = 24 hours
# 144 = 2 days
# 216 = 3 days
# 288 = 4 days
# 360 = 5 days



def make_supervised(df, forecast_columns, sorting_column, input_window=1, output_window=1):
    """
    Converts a multivariate time series dataframe into a supervised learning problem.
    
    Parameters:
    df (pd.DataFrame): The original dataframe with time series data.
    forecast_columns (list): A list of column names to forecast.
    input_window (int): The number of past observations to use as features.
    output_window (int): The number of steps to forecast into the future.
    
    Returns:
    pd.DataFrame: A new dataframe with supervised learning format.
    """
    

    df_new = pd.DataFrame()
    #Put in a for loop here where you iterate over all IDs, to make sure things get correct
    unique_sorts = df[sorting_column].unique()

    forbidden_cols = ["vesselId", "UN_LOCODE", "ISO", "portId", "etaParsed"]
    
    #Iterate through all IDs
    print("Creating supervised data")
    for i, sorts in enumerate(unique_sorts):
        
        loadBar.load_bar(len(unique_sorts),i+1)
        df_supervised = pd.DataFrame()
        sort_df = df[df[sorting_column] == sorts]

        #Iterate through all columns for input features
        for col in sort_df.columns: 
            if col in forbidden_cols:
                    continue
            for i in range(input_window, 0, -1):
                df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
            
            df_supervised[f"{col}_t"] = sort_df[col]
            

        # Create columns for forecast (target) with forward shift
        for col in forecast_columns:
            for j in range(output_window, 0,-1):
                df_supervised[f"{col}_t+{j}"] = sort_df[col].shift(-j)
        
        df_new = pd.concat([df_new, df_supervised])
    
    return df_new

# total_df = pd.read_csv("../../build_resources/data_resampled_20min_markov.csv")

total_df = make_supervised(total_df, OUTPUT_FORECAST, "vesselId", input_window=INPUT_WINDOW, output_window=OUTPUT_WINDOW)


Creating supervised data
[====================] 100.0% complete


In [ ]:

# Display the first few rows of the dataframe
total_df.to_csv(RESOURCE_FOLDER+"/data_resampled_20min_markov_supervised.csv")
print(total_df.head())

              time_t-4             time_t-3             time_t-2  \
0                 None                 None                 None   
1                 None                 None                 None   
2                 None                 None  2024-01-01 00:00:00   
3                 None  2024-01-01 00:00:00  2024-01-01 00:20:00   
4  2024-01-01 00:00:00  2024-01-01 00:20:00  2024-01-01 00:40:00   

              time_t-1               time_t  cog_t-4  cog_t-3  cog_t-2  \
0                 None  2024-01-01 00:00:00      NaN      NaN      NaN   
1  2024-01-01 00:00:00  2024-01-01 00:20:00      NaN      NaN      NaN   
2  2024-01-01 00:20:00  2024-01-01 00:40:00      NaN      NaN    284.0   
3  2024-01-01 00:40:00  2024-01-01 01:00:00      NaN    284.0    284.0   
4  2024-01-01 01:00:00  2024-01-01 01:20:00    284.0    284.0    284.0   

   cog_t-1  cog_t  ...  NE_t-3  NE_t-2  NE_t-1  NE_t  latitude_t+1  \
0      NaN  284.0  ...     NaN     NaN     NaN   0.0    -34.627229   
1    2

In [9]:


#Sorting columns
def sort_columns(df):
    
    # Extract suffixes and assign _t as _t0
    columns_with_suffix = []
    for col in df.columns:
        match = re.search(r"_t([+-]?\d*)$", col)
        # If there's no number after _t, treat it as _t0
        suffix = int(match.group(1)) if match.group(1) else 0
        columns_with_suffix.append((col, suffix))
    
    # Sort by suffix value (ascending)
    sorted_t_columns = [col for col, _ in sorted(columns_with_suffix, key=lambda x: x[1])]
    
    # Reorder dataframe columns
    return df[sorted_t_columns]


total_df = total_df.sort_index(ascending = True)
total_df=sort_columns(total_df)

print(total_df)



         latitude_t+1  longitude_t+1  cog_t+1  sog_t+1  rot_t+1  heading_t+1
0          -34.627229     -57.966135    284.0      0.7      0.0         88.0
1          -34.510757     -58.080969    284.0      0.7      0.0         88.0
2          -34.394286     -58.195804    284.0      0.7      0.0         88.0
3          -34.277814     -58.310639    284.0      0.7      0.0         88.0
4          -34.239269     -58.315756    284.0      0.7      0.0         88.0
...               ...            ...      ...      ...      ...          ...
5899716     37.110410     144.761650     58.2     15.7      0.0         51.0
5899717     37.157450     144.851640     57.7     15.4      0.0         50.0
5899718     37.204090     144.947320     58.7     15.1      0.0         48.0
5899719     37.222310     144.984520     59.8     14.9      0.0         48.0
5899720           NaN            NaN      NaN      NaN      NaN          NaN

[5899721 rows x 6 columns]


In [10]:


def train_test_split(df, perc1, perc2, output_window):
    y_list = []
    for j in range(output_window):
        for col in OUTPUT_FORECAST:
            y_list.append(f"{col}_t+{j+1}")
    ys = df[y_list]
    Xs = df.drop(columns = y_list)

    X_train = Xs.iloc[:int(np.round(Xs.shape[0]*perc1)),:]
    y_train = ys.iloc[:int(np.round(Xs.shape[0]*perc1)),:]
    X_val = Xs.iloc[int(np.round(Xs.shape[0]*perc1)):int(np.round(Xs.shape[0]*perc2)),:]
    y_val = ys.iloc[int(np.round(Xs.shape[0]*perc1)):int(np.round(Xs.shape[0]*perc2)),:]
    X_test = Xs.iloc[int(np.round(Xs.shape[0]*perc2)):,:]
    y_test = ys.iloc[int(np.round(Xs.shape[0]*perc2)):,:]

    return X_train, y_train, X_val, y_val, X_test, y_test



X_train, y_train, X_val, y_val, X_test, y_test = train_test_split(total_df, 0.75, 0.85, OUTPUT_WINDOW)



In [11]:
def evaluate(stepsize, preds, y_val):
    print("/"+"-"*50+"\\")
    print("Evaluating model with stepsize", stepsize)

    results = {
        "MAE": mean_absolute_error(y_val, preds),
        "MSE": np.square(np.subtract(y_val,preds)).mean(),
        "R2 Score": r2_score(y_val, preds),
        "RMSE": np.sqrt(np.square(np.subtract(y_val,preds)).mean())
    }

    for metric, value in results.items():
        print(f"{metric}: {value}")
    print("\\"+"-"*50+"/")



In [12]:

print(X_train)



dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest_X = xgb.DMatrix(X_test)

params = {"objective": "reg:squarederror",
            "max_depth": 5,
            "booster": "gbtree",
            "tree-method": "gpu_hist",
            "col_sample_bynode": 0.5,
            "num_parallel_tree": 100,
            "subsample": 0.8,
            "learning_rate": 1,
            #"n_estimators": 100,
            #"reg_alpha": 0.1,
            #"reg_lambda": 0.1,
            #"n_jobs": -1,
            "verbosity": 1
            }

num_boost_round = 5

early_stopping_rounds = 2

print(dtrain)

model = xgb.train(params, dtrain, num_boost_round, evals=[(dval, "validation")], early_stopping_rounds=early_stopping_rounds, verbose_eval=True)


preds = model.predict(dtest_X)



Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[4424791 rows x 0 columns]


XGBoostError: [15:39:04] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\data\data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.

In [ ]:
def closest_n_min_mark(timestamp, n=1):
    timestamp = pd.to_datetime(timestamp)
    minutes = timestamp.minute
    closest_mark = round(minutes / n*20) * n*20
    if closest_mark == 60:
        rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0) + pd.Timedelta(hours=1)
    else:
        rounded_timestamp = timestamp.replace(minute=closest_mark, second=0, microsecond=0)
    
    return rounded_timestamp

In [ ]:
def shift_to_back(process_df):      
    for _, col in enumerate(OUTPUT_FORECAST):

        max_suffix_neg = 0
        max_suffix_pos = 0
        
        # Identify existing suffixes in the process_df for the current column
        while f"{col}_t-{max_suffix_neg+1}" in process_df.columns:
            max_suffix_neg += 1
        while f"{col}_t+{max_suffix_pos+1}" in process_df.columns:
            max_suffix_pos += 1
        for shift in range(max_suffix_neg - 1, -max_suffix_pos, -1):  # Start from max_suffix-1 down to 0
            if shift == 0:
                # Set the new predicted value as the most recent
                process_df[f"{col}_t"] = process_df[f"{col}_t+1"]
            elif shift == 1:
                # Shift the column
                process_df[f"{col}_t-{shift}"] = process_df[f"{col}_t"]
            elif shift > 1:
                # Shift the column
                process_df[f"{col}_t-{shift}"] = process_df[f"{col}_t-{shift - 1}"]
            else:
                process_df[f"{col}_t+{-shift}"] = process_df[f"{col}_t+{-shift + 1}"]

        for shift in range(1, max_suffix_pos+1):
            process_df = process_df.drop(columns=[f"{col}_t+{shift}"])
    
    return process_df


In [ ]:
def predict_far_future(model, features, test_df,  forecast_columns):
    
    X_test = features.copy().iloc[-1:]
    preds = pd.DataFrame(columns=["vesselId", "approximate_time"])
    
    # Determine the furthest time in 20-minute intervals
    furthest_time = closest_n_min_mark(test_df["time"].max())
    current_time = closest_n_min_mark(X_test.index.max())
    
    # Generate the future time steps at 20-minute intervals
    future_steps = pd.date_range(start=current_time, end=furthest_time, freq='20min')
    
    for future_time in future_steps:
        y_pred = model.predict(X_test)

        new_row = pd.DataFrame({
            "vesselId": [test_df["vesselId"].iloc[0]],
            "approximate_time": [future_time]
        })
        for idx, col in enumerate(forecast_columns):
            new_row[f"{col}"] = y_pred[0, idx]  # Use the predicted value
        new_row = markov.add_as_columns(new_row)
        
        
        preds = pd.concat([preds, new_row], ignore_index=True)
        
        # Update X_test for the next iteration
        for idx, col in enumerate(forecast_columns):

            max_suffix = 0
            
            # Identify existing suffixes in the X_test for the current column
            while f"{col}_t-{max_suffix+1}" in X_test.columns:
                max_suffix += 1
            for shift in range(max_suffix - 1, -1, -1):  # Start from max_suffix-1 down to 0
                if shift == 0:
                    # Set the new predicted value as the most recent
                    X_test[f"{col}_t"] = y_pred[0, idx]
                elif shift == 1:
                    # Shift the column
                    X_test[f"{col}_t-{shift}"] = X_test[f"{col}_t"]
                else:
                    # Shift the column
                    X_test[f"{col}_t-{shift}"] = X_test[f"{col}_t-{shift - 1}"]
    
    return preds


csv_parser = CSVParser(folderpath=RESOURCE_FOLDER)

test_df = csv_parser.retrieve_test_data()




In [ ]:
def predict_times(model,total_df,test_df):
    unique_sorts = test_df["vesselId"].unique()
    preds_df = pd.DataFrame()
    result = pd.DataFrame()

    for sorts in unique_sorts:
        latest_features=total_df[total_df["vesselId"] == sorts]
        test_by_vessel_df = test_df[test_df["vesselId"] == sorts]
        latest_features = make_supervised(latest_features, OUTPUT_FORECAST, "vesselId" , INPUT_WINDOW, OUTPUT_WINDOW)
        latest_features = shift_to_back(latest_features)
        latest_features = sort_columns(latest_features)
        preds = predict_far_future(model, latest_features, test_by_vessel_df, OUTPUT_FORECAST)
        preds_df = pd.concat([preds_df, preds])
    
    for test in test_df.iterrows():
        test=pd.Series(test[1])
        new_row = pd.DataFrame()
        new_row=preds_df[
            (preds_df["vesselId"] == test["vesselId"]) & 
            (preds_df["approximate_time"] == closest_n_min_mark(test["time"]))
            ][["latitude", "longitude"]]
        new_row["ID"] = test["ID"]
        new_row["time"] = test["time"]
        
        result = pd.concat([result, new_row])
    result["latitude_predicted"] = result["latitude"]
    result["longitude_predicted"] = result["longitude"]

    return result[["ID","longitude_predicted","latitude_predicted"]]

print(test_df)
result_df = predict_times(model, total_df, test_df)
print(result_df)





In [ ]:
#turn results into a csv file
result_df.to_csv(RESOURCE_FOLDER+"/results.csv", index=False)

In [ ]:
_ = plot_importance(model, height=0.9)

### First model:
Included navstat and etaParsed

Timewindow: (3,2)

MAE: 0.8521843262281953 

MSE: longitude_t+1    21.225563

latitude_t+1      1.993130

longitude_t+2    38.471488

latitude_t+2      3.840146

dtype: float64

R2 Score: 0.9958523607729776

RMSE: longitude_t+1    4.607121

latitude_t+1     1.411783

longitude_t+2    6.202539

latitude_t+2     1.959629

dtype: float64


### Second model:

Added cog, rot and heading to target features.

Timewindow: (3,2)

MAE: 7.198335594601071
MSE: latitude_t+1        1.980426
longitude_t+1      21.577318
cog_t+1          1820.208937
rot_t+1            92.532501
heading_t+1      1172.604934
latitude_t+2        3.813640
longitude_t+2      39.218475
cog_t+2          2370.325440
rot_t+2           107.991661
heading_t+2      1769.347459
dtype: float64
R2 Score: 0.8826565909012996
RMSE: latitude_t+1      1.407276
longitude_t+1     4.645139
cog_t+1          42.663907
rot_t+1           9.619382
heading_t+1      34.243320
latitude_t+2      1.952854
longitude_t+2     6.262466
cog_t+2          48.685988
rot_t+2          10.391904
heading_t+2      42.063612
dtype: float64
